In [1]:
from utils.configuration import get_config_from_json
from utils.training_utilities import set_GPU
from utils.plotting_traces import plot_traces
from seq2point.seq2point import SEQ2POINT
from dataset_management.dataloader import Seq2PointDataLoader
import builtins

In [2]:
builtins.MODEL_CONFIG = get_config_from_json(description="Model Parameters", config_file="configs/model_config.json")
builtins.DATASET_CONFIG = get_config_from_json(description="Dataset Management", config_file="configs/dataset_config.json")
builtins.PLOT_CONFIG = get_config_from_json(description="Plot Settings", config_file="configs/plot_config.json")
builtins.TRAINING_CONFIG = get_config_from_json(description="Training Configuration", config_file="configs/training_config.json")

In [3]:
TRAINING_CONFIG

{'TARGET_APPLIANCE': 'washingmachine',
 'TARGET_HOUSES': {'TRAIN': [2], 'VALIDATE': [2], 'TEST': [2]},
 'SPLIT_PROPORTION': {'TRAIN_PERCENT': 0.6,
  'VALIDATE_PERCENT': 0.2,
  'TEST_PERCENT': 0.2},
 'SUBSET_DAYS': 1,
 'NORMALIZE': 'Standard',
 'TRAIN_BATCH_SIZE': 512,
 'VALIDATION_BATCH_SIZE': 64,
 'TEST_BATCH_SIZE': 64,
 'LEARNING_RATE': 0.0001,
 'NUM_EPOCHS': 2,
 'OPTIMIZER': 'optim.Adam',
 'LOSS': 'nn.MSELoss',
 'LOSS_REDUCTION': 'mean',
 'EARLY_STOPPING_THRESHOLD': 5,
 'PRE_TRAINED_MODEL_FLAG': False,
 'SAVE_MODEL': 'models/',
 'LOAD_MODEL': '',
 'DESCRIPTION': 'Training Configuration'}

In [4]:
import json
import os

path = f'experiments/{TRAINING_CONFIG["TARGET_APPLIANCE"]}/{TRAINING_CONFIG["TARGET_HOUSES"]["TRAIN"]}/'
if not os.path.exists(path):
    os.makedirs(path)
with open(f'{path}/experiment_config.json', 'w') as json_file:
    json.dump(TRAINING_CONFIG, json_file)

In [5]:
dataloaders = Seq2PointDataLoader(
    target_appliance=TRAINING_CONFIG['TARGET_APPLIANCE'], 
    target_houses= TRAINING_CONFIG['TARGET_HOUSES'], 
    proportion = TRAINING_CONFIG['SPLIT_PROPORTION'], 
    subset_days = TRAINING_CONFIG['SUBSET_DAYS'], 
    normalize_with=TRAINING_CONFIG['NORMALIZE'])

Followings are the refit_loader configuration 
{'DATA_FOLDER': 'data/refit/', 'DATA_TYPE': '.csv', 'README_FILE': 'refit_loader/REFIT_Readme.txt', 'REFIT_HOUSES': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]}

Loading specified buildings: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]
Parsing the readme file specified: refit_loader/REFIT_Readme.txt
Loading data for appliance WASHINGMACHINE ...
Fetching WASHINGMACHINE data for House 2
Resampling for house number:  2
Updating data with resampled dataset...
Subetting dataset with 1 days of most activities for House 2
Estimating active durations of: washingmachine
Updating data with selected active appliance activities...
Updating splits with specified proportion from every target house...
Normalization is being performed for training a model. Scalers will be computed/fit considering the train_split and using those scalers, all splits will be normalized/transformed.
Updating splits with n

In [6]:
network = SEQ2POINT().to(set_GPU())
results = network.run(dataloaders.train_dataloader, dataloaders.validation_dataloader, assess_training=False)


Initializing SEQ2POINT model archiecture

Followings are the Model Parameters of your network architecture..
{'CONV_KERNEL': [10, 8, 6, 5, 5],
 'CONV_LAYERS': 5,
 'CONV_PADDING': 0,
 'CONV_STRIDE': 1,
 'DESCRIPTION': 'Model Parameters',
 'INPUT_CHANNELS': [1, 30, 30, 40, 50],
 'LEFT_PAD': [4, 3, 2, 2, 2],
 'LINEAR_INPUT': [29950, 1024],
 'LINEAR_LAYERS': 2,
 'LINEAR_OUTPUT': [1024, 1],
 'OUTPUT_CHANNELS': [30, 30, 40, 50, 50],
 'POOL_KERNEL': [],
 'POOL_STRIDE': [],
 'RIGHT_PAD': [5, 4, 3, 2, 2],
 'SEQUENCE_LENGTH': 599}

SEQ2POINT model archiecture has been initialized.


Followings are the Training Configuration of your experiment..
{'DESCRIPTION': 'Training Configuration',
 'EARLY_STOPPING_THRESHOLD': 5,
 'LEARNING_RATE': 0.0001,
 'LOAD_MODEL': '',
 'LOSS': 'nn.MSELoss',
 'LOSS_REDUCTION': 'mean',
 'NORMALIZE': 'Standard',
 'NUM_EPOCHS': 2,
 'OPTIMIZER': 'optim.Adam',
 'PRE_TRAINED_MODEL_FLAG': False,
 'SAVE_MODEL': 'models/',
 'SPLIT_PROPORTION': {'TEST_PERCENT': 0.2,
            

In [7]:
test_network = SEQ2POINT().to(set_GPU())


Initializing SEQ2POINT model archiecture

Followings are the Model Parameters of your network architecture..
{'CONV_KERNEL': [10, 8, 6, 5, 5],
 'CONV_LAYERS': 5,
 'CONV_PADDING': 0,
 'CONV_STRIDE': 1,
 'DESCRIPTION': 'Model Parameters',
 'INPUT_CHANNELS': [1, 30, 30, 40, 50],
 'LEFT_PAD': [4, 3, 2, 2, 2],
 'LINEAR_INPUT': [29950, 1024],
 'LINEAR_LAYERS': 2,
 'LINEAR_OUTPUT': [1024, 1],
 'OUTPUT_CHANNELS': [30, 30, 40, 50, 50],
 'POOL_KERNEL': [],
 'POOL_STRIDE': [],
 'RIGHT_PAD': [5, 4, 3, 2, 2],
 'SEQUENCE_LENGTH': 599}

SEQ2POINT model archiecture has been initialized.



In [8]:
test_network.inference(dataloaders.test_dataloader)

Loading the model...models/2022-12-02_best_loss_0.pt
Model's state_dict:
conv.1.weight 	 torch.Size([30, 1, 10])
conv.1.bias 	 torch.Size([30])
conv.4.weight 	 torch.Size([30, 30, 8])
conv.4.bias 	 torch.Size([30])
conv.7.weight 	 torch.Size([40, 30, 6])
conv.7.bias 	 torch.Size([40])
conv.10.weight 	 torch.Size([50, 40, 5])
conv.10.bias 	 torch.Size([50])
conv.13.weight 	 torch.Size([50, 50, 5])
conv.13.bias 	 torch.Size([50])
dense.0.weight 	 torch.Size([1024, 29950])
dense.0.bias 	 torch.Size([1024])
dense.2.weight 	 torch.Size([1, 1024])
dense.2.bias 	 torch.Size([1])
Loss is set with loss_reduction = mean
Average Test Loss : 5.767048199387157, Time consumption: 0:00:00.944559s


[0.905463457107544,
 0.8753358125686646,
 1.051973581314087,
 0.9404566884040833,
 1.123887062072754,
 0.26275742053985596,
 0.3428201675415039,
 1.5158991813659668,
 3.499354362487793,
 4.534339904785156,
 5.4660444259643555,
 88.50533294677734,
 8.543002128601074,
 5.050264835357666,
 2.568528652191162,
 3.182976722717285,
 3.5965232849121094,
 3.0654170513153076,
 1.8106902837753296,
 1.3490136861801147,
 1.2208225727081299,
 2.4226913452148438,
 1.6732611656188965,
 1.2661677598953247,
 1.1077702045440674,
 1.6991021633148193,
 2.5104105472564697,
 1.9374276399612427,
 3.224032402038574,
 4.644961833953857,
 5.762399196624756,
 6.863101005554199,
 9.782821655273438,
 13.774587631225586]